Author: Madhusudhanan Balasubramanian, Ph.D.
        The University of Memphis

Ran with the debug config; tested with 3 classes; seems to work.  Let me run it for longer.

June 21, 2024:
    * Copy of Copy22-3 (20240616_1)
    * PRIOR_PROB: 0.005 (from 0.01)
    July 12, 2024:
        * In Inspect_AP_vs_TH_segm.ipynb, AP/AR is not affected by the INFERENCE_TH_TEST values.  Checking visually

June 15, 2024:
    * Copy22-2 copy of Copy22-1
    * BATCH_SIZE_PER_IMAGE: 600
    * POSITIVE FRACTION: 0.5
    * Initial weights: VovNet weights

June 13, 2024:
    * Copy22-1 copy of Copy20-1
    * MIN_SIZE_TRAIN = [1024]
    * MIN_SIZE_TEST = [1024]
    * SIZES_OF_INTEREST = [20, 24, 28, 36]

June 05, 2024:
    * Copy21-1 copy of Copy20-1
    * MIN_SIZE_TRAIN = [512, 768, 1024]
    
May 15, 2024: Copy19-1 copy of Copy18-1:
    * Trying with pixel mean and std from the phase3 axon training dataset
    
April 19, 2024: Copy18-1 copy of Copy17-2:
    * Initialize with model_final_10Oct2024.pth - does this retain the higher AP on both training & validation for Necrotic & Live classes?

April 04, 2024: Copy17-2 copy of Copy17-1:
    * Benefit of higher warmup period or lowering the LR?
    * Higher FCOS.INFERENCE_TH_TEST & TRAIN 0.05 to 0.7
    * Fixed figures: utils/axon_visualizer.py & axon_colormap.py; evaluation/axon_evaluation.py & axon_evaluator.py
    
April 04, 2024: Copy of 16 - testing NUM_CLASSES = 2

April 03, 2024: Copy of 16 - adjusting parameters as in Copy4

Mar 28, 2024: Copy of 15 - running for same iterations with 3 classes instead of 2

Mar 04, 2024: Copy 12 copy of Copy 11 - refining weights with max_dets_per_image parameter for COCOEvaluator in the Trainer class (27Jan2024.pth weights gave lowers performance)

Jan 28, 2024: Copy 11 copy of Copy 10 - more simplified, running with Detectron2 ver. 0.6 (able to specify max_dets_per_image during evaluation)

Jan 27, 2024: Copy10 copy of Copy 9 - rerunning to establish a starting point to resume this project

Nov 30, 2023: Copy9 copied from Copy7 - modified to resume training from the final Oct 10 model "model_final_10Oct2023.pth"

In [1]:
# Work around to enhance error display involving unicode characters
%colors nocolor
%xmode plain

Exception reporting mode: Plain


In [2]:
# import various libraries
import logging
import os, re
import numpy as np
import cv2
import matplotlib.pyplot as plt
#
import torch
#
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances, load_coco_json
#
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.engine import default_argument_parser, default_setup, hooks, launch
#
#MB: 03/31/2024
#from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from centermask.evaluation import COCOEvaluator, inference_on_dataset
#
#from detectron2.utils.visualizer import Visualizer, ColorMode
from centermask.utils.axon_visualizer import Visualizer, ColorMode
#
from centermask.config import get_cfg

Trainer Class

In [3]:
class Trainer(DefaultTrainer):
    """
    This is the same Trainer except that we rewrite the
    `build_train_loader` method.
    """

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        """
        Create evaluator(s) for a given dataset.
        This uses the special metadata "evaluator_type" associated with each builtin dataset.
        For your own dataset, you can simply create an evaluator manually in your
        script and do not have to worry about the hacky if-else logic here.
        """
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        evaluator_list = []
        evaluator_list.append(COCOEvaluator(dataset_name, output_dir=output_folder, max_dets_per_image=600))
        return evaluator_list[0]

Create config and perform basic steps

In [ ]:
cfg = get_cfg()
cfg.merge_from_file("configs/AxonClass/axon_V_39_eSE_FPN_ms_3x.yaml")
cfg.num_gpus = 1
cfg.resume = True
cfg.freeze()
default_setup(cfg, '')

[07/25 10:15:20 detectron2]: Rank of current process: 0. World size: 1
[07/25 10:15:21 detectron2]: Environment info:
----------------------  ---------------------------------------------------------------------------------------------------
sys.platform            linux
Python                  3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:21) [GCC 9.4.0]
numpy                   1.21.6
detectron2              0.6 @/opt/anaconda3/envs/venv_mamba/envs/venv_py37_D2_6a/lib/python3.7/site-packages/detectron2
Compiler                GCC 7.3
CUDA compiler           CUDA 11.3
detectron2 arch flags   3.7, 5.0, 5.2, 6.0, 6.1, 7.0, 7.5, 8.0, 8.6
DETECTRON2_ENV_MODULE   <not set>
PyTorch                 1.10.0 @/opt/anaconda3/envs/venv_mamba/envs/venv_py37_D2_6a/lib/python3.7/site-packages/torch
PyTorch debug build     False
GPU available           Yes
GPU 0,1                 Quadro RTX 8000 (arch=7.5)
Driver version          460.32.03
CUDA_HOME               /usr/local/cuda
Pill

Register COCO datasets

In [5]:
#Register training axon datasets
name = "Phase3_Training"
json_file = "./datasets/Phase3_Training/Phase3_Training.json"
image_root = "./datasets/Phase3_Training"
register_coco_instances(name, {}, json_file, image_root)

Phase3_Tr_metadata = MetadataCatalog.get("Phase3_Training")
print(f"Phase3_Tr_metadata:{Phase3_Tr_metadata}")
Phase3_Tr_dicts = DatasetCatalog.get("Phase3_Training")

#Register validation axon datasets
name = "Phase3_Validation"
json_file = "./datasets/Phase3_Validation/Phase3_Validation.json"
image_root = "./datasets/Phase3_Validation"
register_coco_instances(name, {}, json_file, image_root)

Phase3_Tr_metadata:Metadata(evaluator_type='coco', image_root='./datasets/Phase3_Training', json_file='./datasets/Phase3_Training/Phase3_Training.json', name='Phase3_Training')
[07/24 19:20:03 d2.data.datasets.coco]: Loaded 30 images in COCO format from ./datasets/Phase3_Training/Phase3_Training.json


Training

In [10]:
print(f"cfg.MODEL.FCOS.NMS_TH:{cfg.MODEL.FCOS.NMS_TH}; cfg.MODEL.FCOS.INFERENCE_TH_TEST:{cfg.MODEL.FCOS.INFERENCE_TH_TEST}")
print(f"cfg.MODEL.FCOS.INFERENCE_TH_TRAIN:{cfg.MODEL.FCOS.INFERENCE_TH_TRAIN}")
print(f"cfg.MODEL.WEIGHTS:{cfg.MODEL.WEIGHTS}")
print(f"cfg.SOLVER.BASE_LR:{cfg.SOLVER.BASE_LR}; cfg.SOLVER.MOMENTUM:{cfg.SOLVER.MOMENTUM}; cfg.SOLVER.GAMMA:{cfg.SOLVER.GAMMA}")

cfg.MODEL.FCOS.NMS_TH:0.3; cfg.MODEL.FCOS.INFERENCE_TH_TEST:0.7
cfg.MODEL.FCOS.INFERENCE_TH_TRAIN:0.3
cfg.MODEL.WEIGHTS:vovnet39_ese_detectron2.pth
cfg.SOLVER.BASE_LR:0.01; cfg.SOLVER.MOMENTUM:0.7; cfg.SOLVER.GAMMA:0.1


In [11]:
#Jan 27, 2024: commented out this section that was written on Nov 30, 2023 to allow for training to resume by changing the max_iter
#Not easity to change the max_iter one the training has started -- will investigate later
#cfg.defrost()
#cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
#cfg.SOLVER.MAX_ITER = 13000
#cfg.freeze()
trainer = Trainer(cfg)
trainer.resume_or_load(resume=True)
trainer.train()

[07/25 10:15:37 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(768, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelP6P7(
      (p6): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (p7): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (bottom_up): VoVNet(
      (stem): Sequential(
        (stem_1/conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (stem_1/norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        (stem_1/relu): R